# 20211125

In [1]:
import torch

x_data = torch.Tensor([[1.0, 1.0],[2.0, 3.0],[3.0, 1.0]])
y_data = torch.Tensor([[3.0],[7.0],[7.0]])

class LinearModel(torch.nn.Module):
    def __init__(self):
        super(LinearModel, self).__init__()
        self.Linear = torch.nn.Linear(2,1)
    
    def forward(self, x):
        y_pred = self.Linear(x)
        return y_pred

model = LinearModel()

criterion = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)
print(criterion, optimizer, sep = '\n\n')

for epoch in range(500):
    y_pred = model(x_data)
    loss = criterion(y_pred, y_data)
    #print(epoch, loss)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

D:\Anaconda\envs\pytorch\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


MSELoss()

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [2]:
print('b=', model.Linear.bias.detach().numpy())
print('w=', model.Linear.weight.detach().numpy()[0])

x_test = torch.Tensor([[4.0,2.0]])
y_test = model(x_test)
print('prediction=:', y_test.data.numpy()[0][0])

b= [0.0811654]
w= [1.9753518 0.9851584]
prediction=: 9.952889


# 20211126

In [3]:
import torch

x_data = torch.Tensor([1.0, 2.0, 3.0])
y_data = torch.Tensor([2.0, 4.0, 6.0])
print(x_data, x_data.detach().numpy(), x_data.grad)

w = torch.Tensor([1.0])
w.requires_grad = True
print(w, w.data, w.grad)
print(w.item(), w.data.item())

def forward(x):
    return x*w

def loss(x, y):
    y_pred = forward(x)
    return (y_pred-y)**2

for epoch in range(10):
    for x, y in zip(x_data, y_data):
        # l = loss(x, y)
        l = (x*w-y)**2
        l.backward()
        #print(x,y,w,x.type(),l,l.type())
        w.data = w.data - 0.01*w.grad.data
        w.grad.data.zero_()
        
print(forward(4))

tensor([1., 2., 3.]) [1. 2. 3.] None
tensor([1.], requires_grad=True) tensor([1.]) None
1.0 1.0
tensor([7.8049], grad_fn=<MulBackward0>)


In [4]:
import torch

a = torch.Tensor([1.0])[0]
a.requires_grad = True

b = torch.Tensor([2.0])[0]
b.requires_grad = True

c = a**2+b**3+12
d = 2*c
e = d-10

print(a,b,c,d,e)

e.backward()

print(a.grad,b.grad)

tensor(1., requires_grad=True) tensor(2., requires_grad=True) tensor(21., grad_fn=<AddBackward0>) tensor(42., grad_fn=<MulBackward0>) tensor(32., grad_fn=<SubBackward0>)
tensor(4.) tensor(24.)


In [5]:
import torch

a = torch.tensor([1.0])
a.requires_grad = True

b=a**3
c=b+1
d=c*7

d.backward()

print(a.data.item())
print(a.grad.item())

print(a,b,c,d)
print(a,a.grad)

1.0
21.0
tensor([1.], requires_grad=True) tensor([1.], grad_fn=<PowBackward0>) tensor([2.], grad_fn=<AddBackward0>) tensor([14.], grad_fn=<MulBackward0>)
tensor([1.], requires_grad=True) tensor([21.])


In [6]:
import torch

x = torch.tensor([[-7,-6,-5,-4,-3,-2,-1,0.0,1,2,3,4,5,6,7]])
x.requires_grad = True

y = x**3
print(y)
y = y.sum()
print(y)

y.backward()

print(*x.grad.numpy()[0])

tensor([[-343., -216., -125.,  -64.,  -27.,   -8.,   -1.,    0.,    1.,    8.,
           27.,   64.,  125.,  216.,  343.]], grad_fn=<PowBackward0>)
tensor(0., grad_fn=<SumBackward0>)
147.0 108.0 75.0 48.0 27.0 12.0 3.0 0.0 3.0 12.0 27.0 48.0 75.0 108.0 147.0


In [7]:
import torch

a = torch.tensor([3.0])
a.requires_grad = True

b = a**2

c = b+1

d = c**2

e = torch.tensor([1.0])
e.requires_grad = True

f = e**2

d.backward()

f.backward()

print(a.grad,e.grad)

tensor([120.]) tensor([2.])


only leaf 变量 can requires_grad
声明requires_grad = True在构建计算图之前
一旦backward，相关的计算图会被解构

tensor进行“计算”是在构建计算图，而不是数值计算
data不会构建计算图
所以纯数值计算（如权值更新时）要用data，避免产生计算图

数值提取时用item()避免产生计算图

# 20211127

In [8]:
import torch

x_set = torch.tensor([-1.0, 2.0, 5.0])
y_set = torch.tensor([-1.0, 5.0, 11.0])

w = torch.tensor([1.0, 0.0])
w.requires_grad = True

total_epoch = 5
learning_rate_coef = 0.016
learning_rate_inte = 0.065

for epoch in range(total_epoch):
    for x, y in zip(x_set, y_set):
        loss = (x * w[0] + w[1] - y)**2
        print(epoch, loss.data)
        loss.backward()
        print(w.data, w.grad.data)
        w.data[0] = w.data[0] - learning_rate_coef * w.grad.data[0]
        w.data[1] = w.data[1] - learning_rate_inte * w.grad.data[1]
        w.grad.data.zero_()
        print(w.data, w.grad.data)

def predict(x):
    return w.data[0] * x + w.data[1]

predict(9.0)

0 tensor(0.)
tensor([1., 0.]) tensor([0., 0.])
tensor([1., 0.]) tensor([0., 0.])
0 tensor(9.)
tensor([1., 0.]) tensor([-12.,  -6.])
tensor([1.1920, 0.3900]) tensor([0., 0.])
0 tensor(21.6225)
tensor([1.1920, 0.3900]) tensor([-46.5000,  -9.3000])
tensor([1.9360, 0.9945]) tensor([0., 0.])
1 tensor(0.0034)
tensor([1.9360, 0.9945]) tensor([-0.1170,  0.1170])
tensor([1.9379, 0.9869]) tensor([0., 0.])
1 tensor(0.0189)
tensor([1.9379, 0.9869]) tensor([-0.5494, -0.2747])
tensor([1.9467, 1.0048]) tensor([0., 0.])
1 tensor(0.0686)
tensor([1.9467, 1.0048]) tensor([-2.6193, -0.5239])
tensor([1.9886, 1.0388]) tensor([0., 0.])
2 tensor(0.0025)
tensor([1.9886, 1.0388]) tensor([-0.1005,  0.1005])
tensor([1.9902, 1.0323]) tensor([0., 0.])
2 tensor(0.0002)
tensor([1.9902, 1.0323]) tensor([0.0505, 0.0253])
tensor([1.9894, 1.0306]) tensor([0., 0.])
2 tensor(0.0005)
tensor([1.9894, 1.0306]) tensor([-0.2251, -0.0450])
tensor([1.9930, 1.0336]) tensor([0., 0.])
3 tensor(0.0016)
tensor([1.9930, 1.0336]) tensor

tensor(18.9818)

In [9]:
import torch

x_data = torch.Tensor([[1.0, 1.0], [2.0, 3.0], [3.0, 1.0], [5.0, 8.0]])
y_data = torch.Tensor([[3.0], [7.0], [7.0], [18.0]])

class LinearModel(torch.nn.Module):
    def __init__(self):
        super(LinearModel, self).__init__()
        self.Linear = torch.nn.Linear(2,1)
    
    def forward(self, x):
        return self.Linear(x)

model = LinearModel()

criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.015)
#print(criterion, optimizer, sep = '\n\n')
#print(model)
#print(model.parameters())
print(model.Linear.weight.data, model.Linear.bias.data) # initial weights and bias

for epoch in range(1000):
    y_pred = model(x_data)
    loss = criterion(y_pred, y_data)
    #print(epoch, loss.data, end = ' | ')
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
print('\n')
print(model.Linear.weight.data)
print(model.Linear.bias.data)

x_test = torch.tensor([[9.0, 9.0], [7.0, -3.0]])
y_pred = model(x_test)
print(y_pred.data)

tensor([[0.1006, 0.2757]]) tensor([-0.1343])


tensor([[1.9981, 1.0006]])
tensor([0.0041])
tensor([[26.9919],
        [10.9887]])


### notes:

initial weight and bias are random

call model(), model.forward(), or model.Linear() are the same

In [ ]:
import torchvision

train = torchvision.datasets.MNIST(root='C:/Users/user/Desktop', train = True, download = True)
test = torchvision.datasets.MNIST(root='C:/Users/user/Desktop', train = False, download = True)

29.6%

In [ ]:
import torch

x_data = torch.Tensor([[1.0, 1.0], [2.0, 3.0], [3.0, 1.0], [2.0, 8.0], [10.0, 3.0]])
y_data = torch.Tensor([[0], [0], [0], [1], [1]])

class LogisticRegressionModel(torch.nn.Module):
    def __init__(self):
        super(LogisticRegressionModel, self).__init__()
        self.Linear = torch.nn.Linear(2,1)
    
    def forward(self, x):
        return torch.sigmoid(self.Linear(x))

model = LogisticRegressionModel()

criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.015)
#print(criterion, optimizer, sep = '\n\n')
#print(model)
#print(model.parameters())
print(model.Linear.weight.data, model.Linear.bias.data) # initial weights and bias

for epoch in range(1000):
    y_pred = model(x_data)
    loss = criterion(y_pred, y_data)
    #print(epoch, loss.data, end = ' | ')
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
print('\n')
print(model.Linear.weight.data)
print(model.Linear.bias.data)

x_test = torch.tensor([[9.0, 9.0], [7.0, -3.0], [2.0, 8.0], [4.0, 4.0]])
y_pred = model(x_test)
print(y_pred.data)

# 20211128

In [ ]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader

class DiabetesDataset(Dataset):
    def __init__(self, filepath):
        xy = np.loadtxt(filepath, delimiter=',', dtype=np.float32)
        print(xy)
        print(xy.shape)
        self.len = xy.shape[0]
        self.x_data = torch.from_numpy(xy[:, :-1])
        self.y_data = torch.from_numpy(xy[:, [-1]])
        print(self.x_data.data.shape)
        print(self.y_data.data.shape)
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    def __len__(self):
        return self.len

dataset = DiabetesDataset('diabetes.csv.gz')
train_loader = DataLoader(dataset=dataset, batch_size=168, shuffle=True)
#print(train_loader)

class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear1 = torch.nn.Linear(8, 31)
        self.linear2 = torch.nn.Linear(31, 6)
        self.linear3 = torch.nn.Linear(6, 1)
        self.relu = torch.nn.ReLU()
        self.sigmoid = torch.nn.Sigmoid()
    def forward(self, x):
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = self.sigmoid(self.linear3(x))
        return x

model = Model()

criterion = torch.nn.BCELoss(size_average=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.003)

for epoch in range(100):
    for i, data in enumerate(train_loader, 0):
        #print(i,data)
        # 1. Prepare data
        inputs, labels = data
        # 2. Forward
        y_pred = model(inputs)
        loss = criterion(y_pred, labels)
        #print(epoch, i, loss.item())
        # 3. Backward
        optimizer.zero_grad()
        loss.backward()
        # 4. Update
        optimizer.step()


# 20211129

https://pytorch.apachecn.org/#/

read this document

In [ ]:
import torch

a = torch.tensor([1.0, 2.0], dtype = torch.float32)
print(a.shape)
print(a.dtype)
print(a.device)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    print(1)
    a = a.to(device)
    
print(a.shape)
print(a.dtype)
print(a.device)

In [ ]:
import torch

a = torch.tensor([1.0, 2.0], dtype = torch.float32)

print(a)

b = a.add(1.0)

print(a, b)

b = a.add_(1.0)

print(a, b)

# 20211209

In [ ]:
import torch

a = torch.tensor([2.0, 5.0])
a.requires_grad = True
c = torch.tensor([3.0, 4.0])
c.requires_grad = True

optimizer = torch.optim.SGD([a, c], lr = 0.01)

for i in range(10):
    optimizer.zero_grad()
    b = sum(a**2 + a + 1)
    e = a
    b.backward()
    #d = sum(c*9)
    #d.backward()
    #print(a.grad.data, a.data, c.grad.data, c.data)
    
    optimizer.step()

    #print(a.grad.data, a.data, c.grad.data, c.data)
    print(a.grad.data, a.data, c.data, e.grad.data, e.data)

# 20211210

### policy network

In [ ]:
import torch

class GCXtest(torch.nn.Module):
    def __init__(self):
        super(GCXtest, self).__init__()
        self.layer1 = torch.nn.Linear(4, 7)
        self.layer2 = torch.nn.Linear(7, 2)
        self.rl = torch.nn.ReLU()
        self.sm = torch.nn.Sigmoid()
        
    def forward(self, x):
        x = self.rl(self.layer1(x))
        x = self.sm(self.layer2(x))
        x = x / x.sum()
        return x
    
model = GCXtest()
optim = torch.optim.SGD(model.parameters(), lr = 0.001)

X = torch.tensor([2.0, 3.0, 5.0, 7.0])

for i in range(100):
    p = model(X)
    if p.data[0] > p.data[1]:
        a = 0
    else:
        a = 1
    if a == 0:
        loss = p * torch.tensor([1, 0])
    if a == 1:
        loss = p * torch.tensor([0, 1])
    loss = loss.sum()
    
    optim.zero_grad()
    loss.backward()
    optim.step()
    
    print(p.data, a)

### value network

In [1]:
import torch
import numpy as np

class GCXtest(torch.nn.Module):
    def __init__(self):
        super(GCXtest, self).__init__()
        self.layer1 = torch.nn.Linear(1, 7)
        self.layer2 = torch.nn.Linear(7, 1)
        self.relu = torch.nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        return x
    
model = GCXtest()
model.layer2.bias.data = torch.tensor([1.0])
optim = torch.optim.SGD(model.parameters(), lr = 0.1, momentum = 0.5)

for i in range(300):
    print(i)
    for t in range(20):
        tt = torch.tensor([float(t)])
        r = 1
        v1 = model(tt)
        v2 = model(tt + 1)
        loss = (v1 - 0.9*v2 - r) ** 2
        print('loss',loss.data)
        
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        with torch.no_grad():
            test = torch.tensor([100.0])
            v = model(test)
            print('test',v)
    

0
loss tensor([1.1850])
test tensor([0.])
loss tensor([0.6845])
test tensor([0.])
loss tensor([0.4558])
test tensor([0.])
loss tensor([1.])
test tensor([0.])
loss tensor([1.])
test tensor([0.])
loss tensor([1.])
test tensor([0.])
loss tensor([1.])
test tensor([0.])
loss tensor([1.])
test tensor([0.])
loss tensor([1.])
test tensor([0.])
loss tensor([1.])
test tensor([0.])
loss tensor([1.])
test tensor([0.])
loss tensor([1.])
test tensor([0.])
loss tensor([1.])
test tensor([0.])
loss tensor([1.])
test tensor([0.])
loss tensor([1.])
test tensor([0.])
loss tensor([1.])
test tensor([0.])
loss tensor([1.])
test tensor([0.])
loss tensor([1.])
test tensor([0.])
loss tensor([1.])
test tensor([0.])
loss tensor([1.])
test tensor([0.])
1
loss tensor([0.0071])
test tensor([0.])
loss tensor([0.7093])
test tensor([0.])
loss tensor([0.6015])
test tensor([0.])
loss tensor([0.6362])
test tensor([4.4443])
loss tensor([0.7106])
test tensor([4.8417])
loss tensor([0.6932])
test tensor([2.2470])
loss tensor(

test tensor([5.7907])
loss tensor([0.1772])
test tensor([5.8070])
loss tensor([0.1758])
test tensor([5.8235])
loss tensor([0.1744])
test tensor([5.8402])
loss tensor([0.1730])
test tensor([5.8568])
loss tensor([0.1717])
test tensor([5.8734])
loss tensor([0.1703])
test tensor([5.8899])
loss tensor([0.1689])
test tensor([5.9064])
loss tensor([0.1676])
test tensor([5.9229])
loss tensor([0.1662])
test tensor([5.9393])
loss tensor([0.1649])
test tensor([5.9556])
loss tensor([0.1636])
test tensor([5.9718])
loss tensor([0.1623])
test tensor([5.9880])
loss tensor([0.1610])
test tensor([6.0041])
11
loss tensor([0.0784])
test tensor([6.0066])
loss tensor([0.0168])
test tensor([6.0104])
loss tensor([0.0240])
test tensor([6.0154])
loss tensor([0.0380])
test tensor([6.0218])
loss tensor([0.0488])
test tensor([6.0294])
loss tensor([0.0579])
test tensor([6.0380])
loss tensor([0.0750])
test tensor([17.6503])
loss tensor([0.0945])
test tensor([29.9489])
loss tensor([0.0920])
test tensor([35.9389])
loss

loss tensor([0.0009])
test tensor([12.4407])
loss tensor([0.0004])
test tensor([11.4637])
loss tensor([0.0002])
test tensor([10.8217])
loss tensor([0.0001])
test tensor([10.4241])
loss tensor([7.3228e-05])
test tensor([10.1941])
loss tensor([5.9319e-05])
test tensor([10.0802])
loss tensor([5.2671e-05])
test tensor([10.0518])
loss tensor([4.7753e-05])
test tensor([10.0909])
loss tensor([4.1525e-05])
test tensor([10.1848])
loss tensor([3.2546e-05])
test tensor([10.3192])
loss tensor([2.1139e-05])
test tensor([10.4745])
loss tensor([9.7668e-06])
test tensor([10.6239])
loss tensor([2.0165e-06])
test tensor([10.7367])
loss tensor([6.1482e-08])
test tensor([10.7855])
loss tensor([2.3167e-06])
test tensor([10.7576])
loss tensor([4.3501e-06])
test tensor([10.6640])
23
loss tensor([0.0169])
test tensor([15.0254])
loss tensor([0.0024])
test tensor([15.7345])
loss tensor([0.0024])
test tensor([14.8079])
loss tensor([0.0014])
test tensor([13.5110])
loss tensor([0.0006])
test tensor([12.3935])
loss

test tensor([11.3695])
loss tensor([6.3884e-05])
test tensor([11.0171])
loss tensor([2.4442e-05])
test tensor([10.7650])
loss tensor([8.7402e-06])
test tensor([10.6050])
loss tensor([2.9500e-06])
test tensor([10.5119])
loss tensor([8.3470e-07])
test tensor([10.4620])
loss tensor([1.1205e-07])
test tensor([10.4370])
loss tensor([1.9922e-08])
test tensor([10.4240])
loss tensor([3.2633e-07])
test tensor([10.4130])
loss tensor([9.2961e-07])
test tensor([10.3960])
loss tensor([1.6822e-06])
test tensor([10.3672])
loss tensor([2.3312e-06])
test tensor([10.3230])
loss tensor([2.5487e-06])
test tensor([10.2634])
loss tensor([2.1514e-06])
test tensor([10.1936])
loss tensor([1.2879e-06])
test tensor([10.1233])
loss tensor([4.4184e-07])
test tensor([10.0648])
loss tensor([3.2833e-08])
test tensor([10.0286])
32
loss tensor([0.0018])
test tensor([11.4648])
loss tensor([0.0002])
test tensor([11.7823])
loss tensor([0.0002])
test tensor([11.5681])
loss tensor([0.0001])
test tensor([11.2158])
loss tenso

loss tensor([5.9931e-06])
test tensor([10.3182])
loss tensor([2.2618e-06])
test tensor([10.2410])
loss tensor([7.8831e-07])
test tensor([10.1922])
loss tensor([2.5355e-07])
test tensor([10.1639])
loss tensor([6.2909e-08])
test tensor([10.1489])
loss tensor([4.5848e-09])
test tensor([10.1414])
loss tensor([7.2041e-09])
test tensor([10.1373])
loss tensor([4.9376e-08])
test tensor([10.1334])
loss tensor([1.2050e-07])
test tensor([10.1274])
loss tensor([2.0434e-07])
test tensor([10.1173])
loss tensor([2.7113e-07])
test tensor([10.1021])
loss tensor([2.8522e-07])
test tensor([10.0820])
loss tensor([2.3286e-07])
test tensor([10.0587])
loss tensor([1.3411e-07])
test tensor([10.0356])
loss tensor([4.2827e-08])
test tensor([10.0168])
loss tensor([2.1837e-09])
test tensor([10.0056])
42
loss tensor([0.0002])
test tensor([10.4538])
loss tensor([1.6021e-05])
test tensor([10.5543])
loss tensor([1.8409e-05])
test tensor([10.4888])
loss tensor([1.0838e-05])
test tensor([10.3799])
loss tensor([4.7487e-

...

test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
250
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor(

loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
260
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor(

test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
271
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor(

test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
282
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
283
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test ten

loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
294
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor([0.])
test tensor([10.0000])
loss tensor(

# 20211212
use gpu

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

X, Y = X.to(device), Y.to(device)
pred = model(X)

save model, load model

In [ ]:
torch.save(model, 'model20211211.pkl')

model = torch.load('model20211211.pkl')

# 20211215

In [1]:
import torch

class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.layer1 = torch.nn.Linear(2,5)
        self.layer2 = torch.nn.Linear(2,5)
        self.con = torch.nn.Linear(15, 1)
        self.relu = torch.nn.ReLU()
        
    def forward(self, a, b, c):
        a = self.relu(self.layer1(a))
        b = self.relu(self.layer2(b))
        c = self.relu(self.layer2(c))
        print(a.data, b.data, c.data)
        q = torch.cat((a, b, c), 0)
        x = self.con(q)
        print(q.data)
        return x
        
model = Model()
a = torch.tensor([1.0, 1.0])
b = torch.tensor([2.0, 3.0])
c = torch.tensor([4.0, 4.0])
r = model(a, b, c)
print(r.data)

tensor([1.2072, 0.0000, 1.0340, 0.0000, 0.0000]) tensor([0.7303, 3.0181, 0.7966, 2.0917, 0.0000]) tensor([0.0244, 4.2500, 1.2350, 3.6804, 0.0000])
tensor([1.2072, 0.0000, 1.0340, 0.0000, 0.0000, 0.7303, 3.0181, 0.7966, 2.0917,
        0.0000, 0.0244, 4.2500, 1.2350, 3.6804, 0.0000])
tensor([0.8134])
